In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
from tqdm import tqdm   
import tensorflow as tf

from data_processing.read_rml import get_attrubuts

import sys
sys.path.append("/var/data/apnea/src/vggish")

import vggish_input, vggish_params, vggish_slim
from vggish.vggish_slim import define_vggish_slim, load_vggish_slim_checkpoint

In [2]:
APNEA_DIR = '/var/data/apnea/mic_dataset_spec/1/'
NO_APNEA_DIR = '/var/data/apnea/mic_dataset_spec/0/'

checkpoint_path = "/var/data/apnea/src/vggish/vggish_model.ckpt"  
pca_params_path = "/var/data/apnea/src/vggish/vggish_pca_params.npz"

In [3]:
# with tf.Graph().as_default(), tf.compat.v1.Session() as sess:
#     pool4_output = define_vggish_slim()

#     checkpoint_path = "/var/data/apnea/src/vggish/vggish_model.ckpt"
#     load_vggish_slim_checkpoint(sess, checkpoint_path)

#     for op in sess.graph.get_operations():
#         print(op.name)

In [3]:
def create_dataset(apnea_dir, no_apnea_dir):
    apnea_files = os.listdir(apnea_dir)
    no_apnea_files = os.listdir(no_apnea_dir)[:1500]
    spectograms = []
    labels = []

    for apnea_file in apnea_files:
        apnea_data = np.load(apnea_dir + apnea_file, allow_pickle=True).item()
        
        spectograms.append(apnea_data['spectograms'])
        
        labels.append(apnea_data['label'])

    for no_apnea_file in no_apnea_files:
        no_apnea_data = np.load(no_apnea_dir + no_apnea_file, allow_pickle=True).item()
        
        spectograms.append(no_apnea_data['spectograms'])
        
        labels.append(no_apnea_data['label'])
       

    return np.array(spectograms), np.array(labels)
        


In [4]:
X, y = create_dataset(APNEA_DIR, NO_APNEA_DIR)

In [5]:
from tensorflow.keras import layers, models

def create_model(input_shape):
    model = models.Sequential()

    model.add(layers.Input(shape=input_shape))  # input_shape = (17, 12288)

    model.add(layers.TimeDistributed(layers.Dense(256, activation='relu')))
    model.add(layers.TimeDistributed(layers.Dense(128, activation='relu')))

    model.add(layers.Bidirectional(layers.LSTM(7, return_sequences=False)))

    model.add(layers.Dropout(0.4))

    model.add(layers.Dense(64, activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))

    return model

input_shape = (17, 12288)
model = create_model(input_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 17, 256)           3145984   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 17, 128)           32896     
_________________________________________________________________
bidirectional (Bidirectional (None, 14)                7616      
_________________________________________________________________
dropout (Dropout)            (None, 14)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                960       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 3,187,521
Trainable params: 3,187,521
Non-trainable params: 0
______________________________________________

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Форма X_train:", X_train.shape)  #(len, 17, 12288)
print("Форма y_train:", y_train.shape) #(len, )

Форма X_train: (2367, 17, 12288)
Форма y_train: (2367,)


In [8]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=7,       
    restore_best_weights=True 
)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

Epoch 1/50
37/37 [==============================] - 4s 63ms/step - loss: 0.6941 - accuracy: 0.5120 - precision: 0.5061 - recall: 0.2127 - val_loss: 0.6922 - val_accuracy: 0.5051 - val_precision: 0.6316 - val_recall: 0.0403
Epoch 2/50
37/37 [==============================] - 2s 44ms/step - loss: 0.6915 - accuracy: 0.5239 - precision: 0.5192 - recall: 0.3962 - val_loss: 0.6889 - val_accuracy: 0.5659 - val_precision: 0.5880 - val_recall: 0.4597
Epoch 3/50
37/37 [==============================] - 2s 56ms/step - loss: 0.6796 - accuracy: 0.5775 - precision: 0.5759 - recall: 0.5263 - val_loss: 0.6800 - val_accuracy: 0.5304 - val_precision: 0.6613 - val_recall: 0.1376
Epoch 4/50
37/37 [==============================] - 2s 48ms/step - loss: 0.6200 - accuracy: 0.6705 - precision: 0.6859 - recall: 0.6055 - val_loss: 0.5735 - val_accuracy: 0.7348 - val_precision: 0.7701 - val_recall: 0.6745
Epoch 5/50
37/37 [==============================] - 2s 44ms/step - loss: 0.4490 - accuracy: 0.8086 - precisi

In [9]:
predictions = model.predict(X_test)

In [10]:
from sklearn.metrics import recall_score, precision_score, accuracy_score

thresholds = [0.3, 0.5, 0.7, 0.9]

for threshold in thresholds:
    bin_predictions = (predictions > threshold).astype(int)

    recall = recall_score(y_test, bin_predictions)
    precision = precision_score(y_test, bin_predictions)
    accuracy = accuracy_score(y_test, bin_predictions)

    print(f'Порог = {threshold}')
    print("Recall:", recall)
    print("Precision:", precision)
    print("Accuracy:", accuracy)
    print('-'*25)

Порог = 0.3
Recall: 0.8993288590604027
Precision: 0.8729641693811075
Accuracy: 0.8834459459459459
-------------------------
Порог = 0.5
Recall: 0.8590604026845637
Precision: 0.9142857142857143
Accuracy: 0.8885135135135135
-------------------------
Порог = 0.7
Recall: 0.7986577181208053
Precision: 0.9482071713147411
Accuracy: 0.8766891891891891
-------------------------
Порог = 0.9
Recall: 0.6543624161073825
Precision: 0.9798994974874372
Accuracy: 0.8192567567567568
-------------------------
